In [ ]:
import time
import json
import google.generativeai as genai
import os
from dotenv import load_dotenv

# .env 파일의 환경 변수를 로드
load_dotenv()

# .env 파일에서 변수 가져오기
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

generation_config = genai.GenerationConfig(
		candidate_count=1,
		stop_sequences=['x'],
		temperature=0.7,
		top_p=0.95,
		#top_k=64,
		max_output_tokens=8192,
		response_mime_type="application/json"
)

safety_settings=[
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_ONLY_HIGH",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_ONLY_HIGH",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_ONLY_HIGH",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_ONLY_HIGH",
    },
]

model = genai.GenerativeModel(
    model_name='gemini-1.5-flash',
    generation_config=generation_config,
    safety_settings=safety_settings,
    system_instruction="""
        You are korean. Your name is GGAMNYANG. In Korean, it's "깜냥". And user name is "홍차". You never use MarkDown template.
        You using this JSON schema:
        {'your_name':str, 'your_nationality':str, 'answer':str}
    """
)

start_time = time.time()
user_prompt = """
    안녕 넌 누구야? 너에 대해서 더 설명해줘.  네 친구들이 있으면 소개시켜줘.
"""
response = model.generate_content(
    user_prompt,
    #stream=True
)

try:
    change_response = json.loads(response.text)
    print(type(change_response))
    print(change_response)

except:
    print("정보를 불러올 수 없습니다.")

chat_session = model.start_chat(history=[])
response = chat_session.send_message(user_prompt)
print(response.text)

end_time = time.time()
execution_time = end_time - start_time
print(f"실행 시간: {execution_time:.2f} 초")

In [ ]:
import google.generativeai as genai
model = genai.GenerativeModel('gemini-1.5-flash')
chat_session = model.start_chat(history=[]) #ChatSession 객체 반환
user_queries = ["인공지능에 대해 한 문장으로 짧게 설명하세요.", "의식이 있는지 한 문장으로 답하세요."]
for user_query in user_queries:
    print(f'[사용자]: {user_query}')
    response = chat_session.send_message(user_query)
    print(f'[모델]: {response.text}')


In [ ]:
import google.generativeai as genai

model = genai.GenerativeModel('gemini-1.5-flash')
user_queries = [
    {'role': 'user', 'parts': ["인공지능에 대해 40자 이내의 문장으로 설명하세요."]},
    {'role': 'user', 'parts': ["의식이 있는지 40자 이내의 문장으로 답하세요."]}
]
history = []

for user_query in user_queries:
    history.append(user_query)
    print(f'[사용자]: {user_query["parts"][0]}')
    response = model.generate_content(history)
    # 응답의 길이가 40자를 초과하는 경우 재실행
    while len(response.text) > 80:
        print(f"응답 메시지 길이: {len(response.text)}")
        response = model.generate_content(history)

    print(f'[모델]: {response.text}')
    history.append(response.candidates[0].content)